In [3]:
import os
import pandas as pd

data = []

for folder in os.listdir('evaluations'):
  folder_path = os.path.join('evaluations', folder)
  
  if os.path.isdir(folder_path):

    metrics_path = os.path.join(folder_path, 'metrics.csv')
    metrics_df = pd.read_csv(metrics_path)
    
    metrics_df = metrics_df[~metrics_df['class_name'].str.startswith('Mean')]
    
    split_name = folder.split('-')

    row_data = {
      'model': split_name[0].upper()
    }
    row_data['size'] = split_name[2] if split_name[0] == 'clip' else split_name[1]
    row_data['patch_size'] = (split_name[3] if split_name[0] == 'clip' else split_name[2]).replace('patch', '')
    row_data['res'] = split_name[3] if split_name[0] == 'siglip' else ''
    
    for _, row in metrics_df.iterrows():
      class_name = row['class_name']
      f1_score = row['F1 Score']
      row_data[class_name] = f1_score
    
    data.append(row_data)

result_df = pd.DataFrame(data)
result_df = result_df.sort_values(by=['model', 'size']).reset_index(drop=True)

new_header = pd.MultiIndex.from_tuples(
  [('MODEL', 'Name'), ('MODEL', 'Transf. size'), ('MODEL', 'Patch size'), ('MODEL', 'Resolution')] + [('F1 Score', col) for col in result_df.columns[4:]]
)
result_df.columns = new_header
result_df.to_csv('cross_eval_f1.csv', index=False)
result_df



MODEL                                            F1 Score  \
     Name Transf. size Patch size Resolution ANTHONY OF PADUA   
0    CLIP         base         16                       6.12%   
1    CLIP         base         32                       9.32%   
2    CLIP        large         14                      14.21%   
3  SIGLIP         base         16        512           12.83%   
4  SIGLIP        large         16        384           21.78%   
5  SIGLIP       so400m         14        384           13.79%   

                                                                             \
  FRANCIS OF ASSISI  JEROME JOHN THE BAPTIST MARY MAGDALENE    PAUL   PETER   
0             5.22%  51.82%           36.76%         20.15%  30.00%  32.71%   
1            17.65%  51.75%           27.75%         33.50%  22.56%  22.33%   
2            37.21%  59.65%           47.78%         34.08%  32.52%  46.58%   
3            27.42%  54.84%            5.83%         45.14%   3.77%  54.65%   
4            39.42%  52.07%            3.88%         49.30%   0.00%  56.98%   
5             2.74%  64.21%           68.42%         27.06%   7.27%  63.32%   

                                             
  SAINT DOMINIC SAINT SEBASTIAN VIRGIN MARY  
0        25.81%          29.17%      36.43%  
1         8.44%          23.24%      41.32%  
2        23.04%          32.84%      54.12%  
3         0.00%          29.27%      79.06%  
4         3.21%          57.63%      85.82%  
5         3.57%          47.62%      63.97%

In [4]:
import os
import pandas as pd

data = []

for folder in os.listdir('evaluations'):
  folder_path = os.path.join('evaluations', folder)
  
  if os.path.isdir(folder_path):
    metrics_path = os.path.join(folder_path, 'metrics.csv')
    metrics_df = pd.read_csv(metrics_path)
    
    split_name = folder.split('-')
    
    row_data = {
      'model': split_name[0].upper()
    }
    row_data['size'] = split_name[2] if split_name[0] == 'clip' else split_name[1]
    row_data['patch_size'] = (split_name[3] if split_name[0] == 'clip' else split_name[2]).replace('patch', '')
    row_data['res'] = split_name[3] if split_name[0] == 'siglip' else ''
    
    row_data['macro'] = metrics_df.loc[metrics_df['class_name'] == "Mean", 'Average Precision'].values[0]
    row_data['micro'] = metrics_df.loc[metrics_df['class_name'] == "Mean (micro)", 'Average Precision'].values[0]
    
    data.append(row_data)

result_df = pd.DataFrame(data)
result_df = result_df.sort_values(by=['model', 'size']).reset_index(drop=True)

new_header = pd.MultiIndex.from_tuples(
  [('MODEL', 'Name'), ('MODEL', 'Transf. size'), ('MODEL', 'Patch size'), ('MODEL', 'Resolution')] + [('mAP', 'macro'), ('mAP', 'micro')]
)
result_df.columns = new_header
result_df.to_csv('cross_eval_micro-macro.csv', index=False)
result_df

MODEL                                        mAP        
     Name Transf. size Patch size Resolution   macro   micro
0    CLIP         base         16             42.29%  28.40%
1    CLIP         base         32             36.41%  31.76%
2    CLIP        large         14             54.74%  44.17%
3  SIGLIP         base         16        512  42.47%  42.34%
4  SIGLIP        large         16        384  49.57%  50.59%
5  SIGLIP       so400m         14        384  53.81%  42.15%